Evaluate different numbers of requested aspects per prompt

In [1]:
import _sample_reviews

In [2]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_text = _sample_reviews.sample_fyp_433
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))


{'review_ID': 123, 'review_text': 'Great time! played many hours on console first! now trying to get my collection to switch to pc. Highly recommend', 'datetime': datetime.datetime(2024, 3, 21, 20, 29, 40, 18379), 'hash': 'abf1b1bcf2e9fd79ec6a032855ebfa590070c5537419055aee8d2d5d'}



Len of review_text: 20


In [3]:
# define the llm to run with (different LLMs)

from langchain_community.llms import Ollama

llm_mistral7b = Ollama(
    model='mistral:7b-instruct-v0.2-q4_0', temperature=0.4,
    num_gpu = 1,        # disable/enable gpu for testing
)

llm_mixtral8x7b = Ollama(
    model='mixtral:8x7b-instruct-v0.1-q4_0', temperature=0.4,
    num_gpu = 1,        # disable/enable gpu for testing
)

llm_llama2_7b = Ollama(
    model='llama2:7b-chat-q4_0', temperature=0.4, num_gpu = 1
)
# llm_gemma_7b = Ollama()
# llm_gemma_2b = Ollama()

# for looping through the LLM models
Llm_models = {
    'mistral_7b': llm_mistral7b,
    # 'mixtral_8x7b': llm_mixtral8x7b,
    'llama2_7b': llm_llama2_7b
}

In [4]:
# for embedding in testing, we use sentence transformer
# sentence transformer embed text into vector space with dimension 384 for semantic search.

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def custom_len_func(text:str) -> int:
  return len(tokenizer.encode(text, add_special_tokens=False))

text_spliter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
  tokenizer=tokenizer,
  chunk_size=250,
  chunk_overlap=40,
)

docs = text_spliter.create_documents(
  [review_obj['review_text']], metadatas=[{"source":"review_01"}]
)

# define in-memory db and retriever for RAG
db = Chroma.from_documents(documents=docs, embedding=embedding_func, collection_name=review_obj['hash'])
retriever = db.as_retriever(search_kwargs={"k": 5})

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michaelcheng/miniforge3/envs/fyp-test-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(tr

In [5]:
# define callbacks for detecting token usage

from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.outputs.llm_result import LLMResult
from collections import deque

class TokenUsageCallbackHandler(BaseCallbackHandler):

    def __init__(self, deque: deque = None):
        super().__init__()
        self.deque = deque

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        # print(response)

        generation = response.generations[0][0]
        gen_info = generation.generation_info

        # get token usage
        # ref: https://github.com/orgs/langfuse/discussions/1179
        token_usage = gen_info.get('prompt_eval_count', 0) + gen_info.get('eval_count', 0)
        # get time costed (local machine)
        # instead of getting total duration, we get the prompt_eval_duration and eval_duration to exclude the load duration (e.g. to load the model to the gpu, etc.)
        time_costed = gen_info.get('prompt_eval_duration', 1e-10) + gen_info.get('eval_duration', 1e-10)     # in ns, a small value to indicate a inf time when it fails


        # create an object to store the token usage and time costed
        token_usage_obj = {
            'token_usage': token_usage,
            'time_costed': time_costed
        }

        # append the object to the deque
        self.deque.append(token_usage_obj)



common_deque = deque()
chain_config = {
    "callbacks": [TokenUsageCallbackHandler(common_deque)],
}

In [6]:
def combine_token_usage_obj(token_usage_obj_list:list[dict]) -> dict:
    token_usage = 0
    time_costed = 0
    for obj in token_usage_obj_list:
        token_usage += obj['token_usage']
        time_costed += obj['time_costed']

    return {
        'token_usage': token_usage,
        'time_costed': time_costed
    }

---

This notebook focus on different combination of aspects extraction (summarization)

- one aspect per prompt
- (2, 2, 2, 2, 2) (i.e. two aspects per prompt, a total of 5 prompts)
- (3, 3, 4) (i.e. three aspects per prompt for first two, then 4 aspects in last prompt)
- (5, 5) (i.e. five aspects per prompt)

In [7]:
import _prompts

In [8]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [9]:
GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

one aspect per prompt

In [23]:
def prompt_aspect_extraction_peraspect(llm_model_name:str, llm_model, repeat_count=5):
    responses = {}

    for aspect in GAME_ASPECTS:
        responses[str([aspect])] = {}

        rag_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len([aspect]) <= 1 else 'are '}" + ': ' + f'{[aspect]}'
        output_format_tempate = _prompts.OUTPUT_FORMAT_TEMPATE.format(
            aspects_list_01=str([aspect])[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in [aspect]}).replace('\'', '\"')
        )

        relevant_docs = retriever.get_relevant_documents(query=rag_question, k=5)

        for i in range(repeat_count):

            chat_prompt_01 = ChatPromptTemplate.from_messages([
                ("system", _prompts.SYSTEM_TEMPLATE),
                ("user", _prompts.KEYWORD_TEMPLATE_01)
            ])

            chain_01 = chat_prompt_01 | llm_model

            print('The prompt:', chat_prompt_01.format_messages(
                aspects = str([aspect]),
                output_format = output_format_tempate,
                summaries = str('\n'.join([d.page_content for d in relevant_docs]))
            ))

            resp_01 = chain_01.invoke({
                "aspects":str([aspect]),
                "output_format":output_format_tempate,
                "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
            }, config=chain_config)

            token_usage_01 = common_deque.popleft()

            # get token usage result and time costed
            token_usage_result = combine_token_usage_obj([token_usage_01])

            # show result:
            print('-' * 10 + f'Result for {llm_model_name} at attempt={i+1}' + '-' * 10)
            print(resp_01); print()
            print('Token usage:', token_usage_result['token_usage'])
            print('Time costed: {:.04} s'.format(float(token_usage_result['time_costed']) / 1e9))
            print('\n\n')

            responses[str([aspect])][f'attempt_{i}'] = {
                'resp_01': resp_01,
                'token_usage': token_usage_result['token_usage'],
                'time_costed': token_usage_result['time_costed']
            }

    return responses

(3, 3, 4) combination

In [11]:
def prompt_aspect_extraction_334(llm_model_name:str, llm_model, repeat_count=5):

    # aspects_response = {k: '' for k in GAME_ASPECTS}
    responses = {}

    for (start, end) in [(0, 3), (3, 6), (6, 10)]:
        aspects = GAME_ASPECTS[start:end]
        responses[str(aspects)] = {}        # storing the result

        rag_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
        output_format_tempate = _prompts.OUTPUT_FORMAT_TEMPATE.format(
            aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
        )

        relevant_docs = retriever.get_relevant_documents(query=rag_question, k=5)

        for i in range(repeat_count):

            chat_prompt_01 = ChatPromptTemplate.from_messages([
                ("system", _prompts.SYSTEM_TEMPLATE),
                ("user", _prompts.KEYWORD_TEMPLATE_01)
            ])

            chain_01 = chat_prompt_01 | llm_model

            print('The prompt:', chat_prompt_01.format_messages(
                aspects = str(aspects),
                output_format = output_format_tempate,
                summaries = str('\n'.join([d.page_content for d in relevant_docs]))
            ))

            resp_01 = chain_01.invoke({
                "aspects":str(aspects),
                "output_format":output_format_tempate,
                "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
            }, config=chain_config)

            token_usage_01 = common_deque.popleft()

            # get token usage result and time costed
            token_usage_result = combine_token_usage_obj([token_usage_01])

            # show result:
            print('-' * 10 + f'Result for {llm_model_name} at attempt={i+1}' + '-' * 10)
            print(resp_01); print()
            print('Token usage:', token_usage_result['token_usage'])
            print('Time costed: {:.04} s'.format(float(token_usage_result['time_costed']) / 1e9))
            print('\n\n')

            responses[str(aspects)][f'attempt_{i}'] = {
                'resp_01': resp_01,
                'token_usage': token_usage_result['token_usage'],
                'time_costed': token_usage_result['time_costed']
            }

    return responses


(2, 2, 2, 2, 2) prompt

In [12]:
def prompt_aspect_extraction_22222(llm_model_name:str, llm_model, repeat_count=5):

    # aspects_response = {k: '' for k in GAME_ASPECTS}
    responses = {}

    for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
        aspects = GAME_ASPECTS[start:end]
        responses[str(aspects)] = {}        # storing the result

        rag_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
        output_format_tempate = _prompts.OUTPUT_FORMAT_TEMPATE.format(
            aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
        )

        relevant_docs = retriever.get_relevant_documents(query=rag_question, k=5)

        for i in range(repeat_count):

            chat_prompt_01 = ChatPromptTemplate.from_messages([
                ("system", _prompts.SYSTEM_TEMPLATE),
                ("user", _prompts.KEYWORD_TEMPLATE_01)
            ])

            chain_01 = chat_prompt_01 | llm_model

            print('The prompt:', chat_prompt_01.format_messages(
                aspects = str(aspects),
                output_format = output_format_tempate,
                summaries = str('\n'.join([d.page_content for d in relevant_docs]))
            ))

            resp_01 = chain_01.invoke({
                "aspects":str(aspects),
                "output_format":output_format_tempate,
                "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
            }, config=chain_config)

            token_usage_01 = common_deque.popleft()

            # get token usage result and time costed
            token_usage_result = combine_token_usage_obj([token_usage_01])

            # show result:
            print('-' * 10 + f'Result for {llm_model_name} at attempt={i+1}' + '-' * 10)
            print(resp_01); print()
            print('Token usage:', token_usage_result['token_usage'])
            print('Time costed: {:.04} s'.format(float(token_usage_result['time_costed']) / 1e9))
            print('\n\n')

            responses[str(aspects)][f'attempt_{i}'] = {
                'resp_01': resp_01,
                'token_usage': token_usage_result['token_usage'],
                'time_costed': token_usage_result['time_costed']
            }

    return responses


(5, 5) approach

In [13]:
def prompt_aspect_extraction_55(llm_model_name:str, llm_model, repeat_count=5):

    # aspects_response = {k: '' for k in GAME_ASPECTS}
    responses = {}

    for (start, end) in [(0, 5), (5, 10)]:
        aspects = GAME_ASPECTS[start:end]
        responses[str(aspects)] = {}        # storing the result

        rag_question = _prompts.QUESTION_TEMPLATE_01 + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
        output_format_tempate = _prompts.OUTPUT_FORMAT_TEMPATE.format(
            aspects_list_01=str(aspects)[1:-1].replace('\'', '\"'), output_json_template=str({k: '...' for k in aspects}).replace('\'', '\"')
        )

        relevant_docs = retriever.get_relevant_documents(query=rag_question, k=5)

        for i in range(repeat_count):

            chat_prompt_01 = ChatPromptTemplate.from_messages([
                ("system", _prompts.SYSTEM_TEMPLATE),
                ("user", _prompts.KEYWORD_TEMPLATE_01)
            ])

            chain_01 = chat_prompt_01 | llm_model

            print('The prompt:', chat_prompt_01.format_messages(
                aspects = str(aspects),
                output_format = output_format_tempate,
                summaries = str('\n'.join([d.page_content for d in relevant_docs]))
            ))

            resp_01 = chain_01.invoke({
                "aspects":str(aspects),
                "output_format":output_format_tempate,
                "summaries": str('\n'.join([d.page_content for d in relevant_docs]))
            }, config=chain_config)

            token_usage_01 = common_deque.popleft()

            # get token usage result and time costed
            token_usage_result = combine_token_usage_obj([token_usage_01])

            # show result:
            print('-' * 10 + f'Result for {llm_model_name} at attempt={i+1}' + '-' * 10)
            print(resp_01); print()
            print('Token usage:', token_usage_result['token_usage'])
            print('Time costed: {:.04} s'.format(float(token_usage_result['time_costed']) / 1e9))
            print('\n\n')

            responses[str(aspects)][f'attempt_{i}'] = {
                'resp_01': resp_01,
                'token_usage': token_usage_result['token_usage'],
                'time_costed': token_usage_result['time_costed']
            }

    return responses


---

LLAMA2

In [14]:
APPROACHES = ['1111111111', '22222', '334', '55']

In [24]:
llama2_7b_result = {}
llm_model_name = 'llama2_7b'

print(f'Running with {llm_model_name}...')

llama2_7b_result[APPROACHES[0]] = prompt_aspect_extraction_peraspect(llm_model_name, llm_llama2_7b, repeat_count=1)
llama2_7b_result[APPROACHES[1]] = prompt_aspect_extraction_22222(llm_model_name, llm_llama2_7b, repeat_count=1)
llama2_7b_result[APPROACHES[2]] = prompt_aspect_extraction_334(llm_model_name, llm_llama2_7b, repeat_count=1)
llama2_7b_result[APPROACHES[3]] = prompt_aspect_extraction_55(llm_model_name, llm_llama2_7b, repeat_count=1)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Running with llama2_7b...
The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Gameplay\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Gameplay", the JSON should be: {"Gameplay": "..."}\nOnly output the JSON. Do NOT output other text.\n\nGreat time! played many hours on console first! now trying to get my collection to switch to pc. Highly recommend\n\nIf you don\'t know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.')]


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for llama2_7b at attempt=1----------
{"Gameplay": "...Highly recommend"}

Token usage: 9
Time costed: 0.3186 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Narrative\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Narrative", the JSON should be: {"Narrative": "..."}\nOnly output the JSON. Do NOT output other text.\n\nGreat time! played many hours on console first! now trying to get my collection to switch to pc. Highly recommend\n\nIf you don\'t know the 

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


...

----------Result for llama2_7b at attempt=1----------
{"Sound": "...", "Graphics & Art Design": "...", "Performance": "..."}

Token usage: 200
Time costed: 0.6915 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Bug\', \'Suggestion\', \'Price\', \'Overall\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Bug", "Suggestion", "Price", "Overall", the JSON should be: {"Bug": "...", "Suggestion": "...", "Price": "...", "Overall": "..."}\nOnly output the JSON. Do NOT output other t

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for llama2_7b at attempt=1----------
{"Bug": "...", "Suggestion": "...", "Price": "...", "Overall": "Highly recommend"}

Token usage: 219
Time costed: 0.8358 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Gameplay\', \'Narrative\', \'Accessibility\', \'Sound\', \'Graphics & Art Design\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Gameplay", "Narrative", "Accessibility", "Sound", "Graphics & Art Design", the JSON should be: {"Gameplay": "...", "Narrative

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for llama2_7b at attempt=1----------
{"Gameplay": "Great time! played many hours on console first! now trying to get my collection to switch to pc.", "Narrative": "NA", "Accessibility": "NA", "Sound": "NA", "Graphics & Art Design": "NA"}

Token usage: 269
Time costed: 1.403 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Performance\', \'Bug\', \'Suggestion\', \'Price\', \'Overall\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Performance", "Bug", "Sugges

In [28]:
# printing result for different approaches

def print_result(result:dict, approach:str):
    if approach == APPROACHES[0]:
        for aspect in GAME_ASPECTS:
            # can print different attempts by changing the attempt number
            print(result[str([aspect])]['attempt_0']['resp_01'])

    if approach == APPROACHES[1]:
        for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
            aspects = GAME_ASPECTS[start:end]
            print(result[str(aspects)]['attempt_0']['resp_01'])

    if approach == APPROACHES[2]:
        for (start, end) in [(0, 3), (3, 6), (6, 10)]:
            aspects = GAME_ASPECTS[start:end]
            print(result[str(aspects)]['attempt_0']['resp_01'])

    if approach == APPROACHES[3]:
        for (start, end) in [(0, 5), (5, 10)]:
            aspects = GAME_ASPECTS[start:end]
            print(result[str(aspects)]['attempt_0']['resp_01'])

def get_approach_token_usage(result:dict, approach:str):
    if approach == APPROACHES[0]:
        token_usage_list = []
        for aspect in GAME_ASPECTS:
            token_usage_list.append(result[str([aspect])]['attempt_0']['token_usage'])

        return sum(token_usage_list)

    if approach == APPROACHES[1]:
        token_usage_list = []
        for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
            aspects = GAME_ASPECTS[start:end]
            token_usage_list.append(result[str(aspects)]['attempt_0']['token_usage'])

        return sum(token_usage_list)

    if approach == APPROACHES[2]:
        token_usage_list = []
        for (start, end) in [(0, 3), (3, 6), (6, 10)]:
            aspects = GAME_ASPECTS[start:end]
            token_usage_list.append(result[str(aspects)]['attempt_0']['token_usage'])

        return sum(token_usage_list)

    if approach == APPROACHES[3]:
        token_usage_list = []
        for (start, end) in [(0, 5), (5, 10)]:
            aspects = GAME_ASPECTS[start:end]
            token_usage_list.append(result[str(aspects)]['attempt_0']['token_usage'])

        return sum(token_usage_list)
    
def get_approach_time_used(result:dict, approach:str):
    if approach == APPROACHES[0]:
        time_used_list = []
        for aspect in GAME_ASPECTS:
            time_used_list.append(result[str([aspect])]['attempt_0']['time_costed'])

        return sum(time_used_list)

    if approach == APPROACHES[1]:
        time_used_list = []
        for (start, end) in [(0, 2), (2, 4), (4, 6), (6, 8), (8, 10)]:
            aspects = GAME_ASPECTS[start:end]
            time_used_list.append(result[str(aspects)]['attempt_0']['time_costed'])

        return sum(time_used_list)

    if approach == APPROACHES[2]:
        time_used_list = []
        for (start, end) in [(0, 3), (3, 6), (6, 10)]:
            aspects = GAME_ASPECTS[start:end]
            time_used_list.append(result[str(aspects)]['attempt_0']['time_costed'])

        return sum(time_used_list)

    if approach == APPROACHES[3]:
        time_used_list = []
        for (start, end) in [(0, 5), (5, 10)]:
            aspects = GAME_ASPECTS[start:end]
            time_used_list.append(result[str(aspects)]['attempt_0']['time_costed'])

        return sum(time_used_list)
            

In [29]:
print('\n\n\n')
print('-' * 10 + 'Result for llama2_7b' + '-' * 10)
for approach in APPROACHES:
    print(f'Approach: {approach}')
    print_result(llama2_7b_result[approach], approach)
    print('Token usage:', get_approach_token_usage(llama2_7b_result[approach], approach))
    print('Time used: {:.04} s'.format(float(get_approach_time_used(llama2_7b_result[approach], approach)) / 1e9))
    print('\n\n')





----------Result for llama2_7b----------
Approach: 1111111111
{"Gameplay": "...Highly recommend"}
{"Narrative": "..."}
{"Accessibility": "..."}
{"Sound": "...Great time! played many hours on console first! now trying to get my collection to switch to pc."}
{"Graphics & Art Design": "..."}
{"Performance": "...Highly recommend..."}
{"Bug": "..."}
{"Suggestion": "...Highly recommend..."}
{"Price": "..."}
{"Overall": "... Great time! played many hours on console first! now trying to get my collection to switch to pc. Highly recommend ..."}
Token usage: 1476
Time used: 5.016 s



Approach: 22222
{"Gameplay": "...", "Narrative": "..."}
{"Accessibility": "Highly accessible and easy to play for all ages.", "Sound": "Rich and immersive soundtrack adds to the gaming experience."}
{"Graphics & Art Design": "...", "Performance": "..."}
{"Bug": "...", "Suggestion": "..."}
{"Price": "...", "Overall": "..."}
Token usage: 912
Time used: 3.274 s



Approach: 334
{"Gameplay": "Great time! played man

MISTRAL-7B

In [30]:
mistral_7b_result = {}
llm_model_name = 'mistral_7b'

print(f'Running with {llm_model_name}...')

mistral_7b_result[APPROACHES[0]] = prompt_aspect_extraction_peraspect(llm_model_name, llm_mistral7b, repeat_count=1)
mistral_7b_result[APPROACHES[1]] = prompt_aspect_extraction_22222(llm_model_name, llm_mistral7b, repeat_count=1)
mistral_7b_result[APPROACHES[2]] = prompt_aspect_extraction_334(llm_model_name, llm_mistral7b, repeat_count=1)
mistral_7b_result[APPROACHES[3]] = prompt_aspect_extraction_55(llm_model_name, llm_mistral7b, repeat_count=1)

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Running with mistral_7b...
The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Gameplay\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Gameplay", the JSON should be: {"Gameplay": "..."}\nOnly output the JSON. Do NOT output other text.\n\nGreat time! played many hours on console first! now trying to get my collection to switch to pc. Highly recommend\n\nIf you don\'t know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.')]


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for mistral_7b at attempt=1----------
 {"Gameplay": "Users have invested significant time into the gameplay on console and highly recommend it."}

Token usage: 242
Time costed: 0.8235 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Narrative\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Narrative", the JSON should be: {"Narrative": "..."}\nOnly output the JSON. Do NOT output other text.\n\nGreat time! played many hours on console first! now trying to get

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


...

----------Result for mistral_7b at attempt=1----------
 {"Sound": "Review mentions playing many hours with no mention of sound issues or praise.", "Graphics & Art Design": "No explicit mention in review, but no complaints either.", "Performance": "NA"}

Token usage: 226
Time costed: 1.178 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Bug\', \'Suggestion\', \'Price\', \'Overall\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Bug", "Suggestion", "Price", "Overall", the JSO

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for mistral_7b at attempt=1----------
 {"Bug": "NA", "Suggestion": "NA", "Price": "NA", "Overall": "Great time! played many hours on console first!"}

Token usage: 231
Time costed: 1.117 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Gameplay\', \'Narrative\', \'Accessibility\', \'Sound\', \'Graphics & Art Design\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Gameplay", "Narrative", "Accessibility", "Sound", "Graphics & Art Design", the JSON should be: {

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


----------Result for mistral_7b at attempt=1----------
 {"Gameplay": "Provides engaging gameplay experience, played for many hours.", "Narrative": "NA", "Accessibility": "NA", "Sound": "NA", "Graphics & Art Design": "NA"}

Token usage: 262
Time costed: 1.294 s



The prompt: [SystemMessage(content="You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question."), HumanMessage(content='You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.\nThe aspects are [\'Performance\', \'Bug\', \'Suggestion\', \'Price\', \'Overall\']. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.\nOutput the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are "Performance", "Bug", "Suggestion", "Price", "Overall", the JS

In [31]:
print('\n\n\n')
print('-' * 10 + f'Result for {llm_model_name}' + '-' * 10)
for approach in APPROACHES:
    print(f'Approach: {approach}')
    print_result(mistral_7b_result[approach], approach)
    print('Token usage:', get_approach_token_usage(mistral_7b_result[approach], approach))
    print('Time used: {:.04} s'.format(float(get_approach_time_used(mistral_7b_result[approach], approach)) / 1e9))
    print('\n\n')





----------Result for mistral_7b----------
Approach: 1111111111
 {"Gameplay": "Users have invested significant time into the gameplay on console and highly recommend it."}
 {"Narrative": "Players have invested many hours into the game's narrative on console and highly recommend it."}
 {"Accessibility": "The game is enjoyable with many hours of playtime on console. Some users are trying to switch their collection to PC version."}
 {"Sound": "Reviews mention playing the game for many hours on console and now trying to switch to PC. No specific comments about the sound aspect."}

NA
 {"Graphics & Art Design": "The game's graphics have been praised by many reviewers, making the switch from console to PC worthwhile."}
 {"Performance": "Played many hours on console and now trying to get collection to switch to PC, highly recommends it."}
 NA
{"Bug": "The reviews do not provide sufficient information about bugs in the game."}
 {"Suggestion": "The game is highly recommended for playing on b

GEMMA-7B